In [1]:
import torch
from torch import nn
from torch.functional import F
from d2l import d2l

In [2]:
train_iter, vocab = d2l.load_data_time_machine(batch_size=32, num_steps=10, token_type='char')

In [7]:
class Model(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.gru = nn.GRU(self.vocab_size, self.hidden_size)
        self.linear = nn.Linear(self.hidden_size, self.vocab_size)
        
    def forward(self, X, state):
        output, state = self.gru(X, state)
        return self.linear(output), state

In [8]:
hidden_size = 256
net = Model(len(vocab), hidden_size)
loss = nn.CrossEntropyLoss()

trainer = torch.optim.SGD(net.parameters(), lr=0.5)

In [9]:
for epoch in range(100):
    state = torch.zeros(1, 32, hidden_size)
    for X, y in train_iter:
        X = F.one_hot(X.T, len(vocab)).to(torch.float32)
        y = F.one_hot(y.T, len(vocab)).to(torch.float32)
        state.detach_()
        y_hat, state = net(X, state)
    #     print(X.shape, y.shape, y_hat.shape, state.shape)
        l = loss(y_hat, y)
        print('epoch:', epoch + 1, l, end='\r')
        trainer.zero_grad()
        l.backward()
        trainer.step()